In [1]:
from pathlib import Path
from typing import Dict, Tuple
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm import tqdm
import csv
# Specify the directory path
# directory_path = Path("result")
directory_path = Path("../lightning_logs/2024-01-22 08:06:19/")

result_csv_path = Path("results2.csv")

result_dict : Dict[Tuple[int, str], float] = {}

f = open(result_csv_path, "w")
writer = csv.writer(f)
writer.writerow(["iter","model","ofv"])
f.close()

iterations = [entry for entry in directory_path.iterdir() if entry.is_dir()]
for models in tqdm(iterations):
    iteration = int(models.name)
    for model in models.iterdir():
        key = (iteration, model.name)
        logs = next(model.iterdir())
        versions = next(logs.iterdir())
        event = next(versions.glob('event*'))
        event_acc = EventAccumulator(str(event))
        event_acc.Reload()
        scalar_data = event_acc.Scalars("train_loss")[-1].value
        result_dict[key] = float(scalar_data)
        f = open(result_csv_path, "a")
        writer = csv.writer(f)
        writer.writerow([str(iteration), model.name, scalar_data])
        f.close()

100%|██████████| 33/33 [01:17<00:00,  2.34s/it]


In [5]:
from dataclasses import dataclass
import pandas as pd

@dataclass
class OFVs :
    base_model : float
    cov_1_2_3_model : float
    cov_1_2_model :float
    cov_1_3_model : float
    cov_2_3_model : float
    cov_1_model : float
    cov_2_model : float
    cov_3_model : float

    def compare_ofv(self, less_cov_model, more_cov_model):
        assert more_cov_model <= less_cov_model or  more_cov_model - less_cov_model < 3.84, print(more_cov_model-less_cov_model)

    def check_ofv(self):
        self.compare_ofv(self.cov_1_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_2_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_3_model, self.cov_1_2_3_model)

        self.compare_ofv(self.cov_1_2_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_1_3_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_2_3_model, self.cov_1_2_3_model)

        self.compare_ofv(self.cov_1_model, self.cov_1_2_model)
        self.compare_ofv(self.cov_1_model, self.cov_1_3_model)
                         
        self.compare_ofv(self.cov_2_model, self.cov_1_2_model)
        self.compare_ofv(self.cov_2_model, self.cov_2_3_model)

        self.compare_ofv(self.cov_3_model, self.cov_1_3_model)
        self.compare_ofv(self.cov_3_model, self.cov_2_3_model)

        self.compare_ofv(self.base_model, self.cov_1_model)
        self.compare_ofv(self.base_model, self.cov_2_model)
        self.compare_ofv(self.base_model, self.cov_3_model)
    
    # def backward_elimination(self):
    #     step1 = [1,2,3]
    #     step2 = [[2,3],[1,3],[1,2]]
    #     for i in step1:
    #         for j in step2:
                
        


df =pd.read_csv('results2.csv')
df_by_iter = df.groupby('iter')
error_list = []
for iter, d in df_by_iter:
    try :
        ofv_dict = {}
        for _, row in d.iterrows():
            ofv_dict[row['model']] =row['ofv']
        ofvs = OFVs(**ofv_dict)
        ofvs.check_ofv()
    except :
        print(iter)
        error_list.append(iter)

error_list.__len__()

18.484588623046875
11
12.390106201171875
16
4.4657745361328125
24
32


4